# Using Pyrockos Squirrel

After https://pyrocko.org/docs/current/library/examples/squirrel/cli_tool.html#variant-1-quick-and-dirty-script

In [ ]:
import numpy as np
from pyrocko import progress
from pyrocko.util import setup_logging, time_to_str, str_to_time_fillup as stt
from pyrocko.squirrel import Squirrel

In [ ]:
# All data access will happen through a single Squirrel instance:
sq = Squirrel()

## Event catalog

In [ ]:
sq.add_catalog('geofon') # or gcmt

In [ ]:
tmin = stt('2016-10-30 00:00:00')
tmax = stt('2016-10-30 23:50:00')

sq.update(tmin=tmin, tmax=tmax)

In [ ]:
evs = sq.get_events()

In [ ]:
len(evs)

In [ ]:
for ev in evs:
    if ev.magnitude > 6:
        print(ev.magnitude, ev.region)
        
        if ev.region == 'Central Italy':
            tevent = ev.time
            print(tevent)

## Stations plus Channel

In [ ]:
# Add online data source. Enable access to BGR's FDSN web service and restrict
# to GR network and LH channels only:
sq.add_fdsn('bgr', query_args=dict(network='GR', channel='LH?'))

Similar to the query option *level* squirrel has *get_stations()*, *get_channels()* or *get_response()*.

In [ ]:
stas = sq.get_stations(codes='*.*.*')
# stas = sq.get_stations(codes='*.BFO.*')
print(len(stas))

In [ ]:
for sta in stas:
    print(sta)

In [ ]:
chas = sq.get_channels(codes='*.BFO.*.*')
print(len(chas))

In [ ]:
for cha in chas:
    print(cha)
    print(cha)

In [ ]:
resps = sq.get_response(tmin=tmin, tmax=tmax, codes='GR.BFO..LHZ')#.get_effective()
print(resps)

Note that for *get_response()* only one station at once is allowed. Therefor, to get all you stations response, you have to use a loop. To can give the channel object directly, as it will automatically retrieves all informations.

In [ ]:
for cha in chas:
    resps = sq.get_response(cha)#.get_effective()
    print(cha)
    print(resps)

## Waveform

https://pyrocko.org/docs/current/library/reference/squirrel/base.html#pyrocko.squirrel.base.Squirrel.update_waveform_promises

In [ ]:
tmin = tevent + 0
tmax = tevent + 1500
# Ensure meta-data from online sources is up to date for the selected time
# span:
sq.update(tmin=tmin, tmax=tmax)

# Allow waveform download for station BFO. This does not download anything
# yet, it just enables downloads later, when needed. Omit `tmin`, `tmax`,
# and `codes` to enable download of everything selected in `add_fdsn(...)`.
sq.update_waveform_promises(tmin=tmin, tmax=tmax, codes='*.BFO.*.*')

In [ ]:
batches = sq.chopper_waveforms(
        tmin=tmin,
        tmax=tmax,
        codes='*.BFO.*.LH*',
        want_incomplete=False,   # Skip incomplete windows.
        snap_window=True)

In [ ]:
for batch in batches:
    for tr in batch.traces:        
        print(tr)        
        tr.plot()

Restitution of the waveform is also possible, in a rather easy fashion by combining it with *get_response()*.

In [ ]:
# Frequency band for restitution:
fmin = 0.01
fmax = 0.05

# Time length for padding (half of overlap).
tpad = 1.0 / fmin

batches = sq.chopper_waveforms(
        tmin=tmin,
        tmax=tmax,
        codes='*.BFO.*.LH*',
#         tinc=3600.,              # One hour time windows (payload).
#         tpad=tpad,               # Add padding to absorb processing artifacts.
        want_incomplete=False,   # Skip incomplete windows.
        snap_window=True)

for batch in batches:
    for tr in batch.traces:
        resp = sq.get_response(tr).get_effective()
        
        tr = tr.transfer(
            tpad,                  # Fade in / fade out length.
            (0.5*fmin, fmin,
             fmax, 2.0*fmax),      # Frequency taper.
            resp,                  # Complex frequency response of instrument.
            invert=True,           # Use inverse of instrument response.
            cut_off_fading=False)  # Disable internal trimming.
        
        tr.chop(batch.tmin, batch.tmax)
        
        print(tr)        
        tr.plot()
